In [11]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
import torch

<font size=5> Matrix Factorization </font>

In [4]:
# 117715 筆資料
df = pd.read_pickle(r'../data/filtered_reviews_group.pkl')
df = df[["AppID","UserID","Like"]]
df['Interacted'] = 1
df.head()

,AppID,UserID,Like,Interacted
0,945360,76561197996720254,1,1
1,945360,76561198133726836,1,1
2,945360,76561198271396832,1,1
3,945360,76561198083568932,0,1
4,945360,76561198123845513,1,1


In [13]:
def get_trainVector(df, user_train_set):
    business_ids = list(df.columns)
    user_ids = list(df.index)
    return [ [1 if business_id in user_train_set[user_id] else 0 for business_id in business_ids ] for user_id in tqdm(user_ids)]

In [12]:
# construct rating matrix and interaction matrix
interaction_matrix = df.pivot_table(index='UserID', columns='AppID', values='Interacted').fillna(0)
rating_matrix = df.pivot_table(index='UserID', columns='AppID', values='Like').fillna(0)
trainVector = torch.tensor(get_trainVector(interaction_matrix, user_train_set)).to(torch.float32)

NameError: name 'train_test_random_split' is not defined

In [79]:
def matrix_factorization(matrix):
    # n_components is embedding dimension。vervose=1 顯示訓練過程
    model = NMF(n_components=128, init='random', random_state=0, verbose=0)
    user_embeddings = model.fit_transform(trainVector*matrix.values)
    # 轉成 float32 有兩種寫法
    encoded_user_embeddings = np.asarray(user_embeddings, dtype=np.float32)
    app_embeddings = model.components_.T
    encoded_app_embeddings = np.asarray(app_embeddings.astype('float32'))
    print(encoded_user_embeddings.shape)
    user_id_emb = dict( zip(matrix.index, encoded_user_embeddings) )
    app_id_emb = dict( zip(matrix.columns, encoded_app_embeddings) )

    return user_id_emb, app_id_emb

user_id_emb, app_id_emb = matrix_factorization(rating_matrix)

(4304, 128)
